In [35]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

#initialize venue data and filter
venue_data = pd.read_csv("..\Wedding\la_wedding_venue_data.csv")

#update price datatype
venue_data['price']=venue_data['price'].str.replace(",","")
venue_data['price']=pd.to_numeric(venue_data['price'])

#update postal code datatype
venue_data['postal code']=venue_data['postal code'].astype(str)

#trim venue data of irrelevant price values. Values of 1 indicate that no price was submitted to the site
venues = venue_data[venue_data['price']>1]

#append "US" to postal code for geocoding
venues = venues.copy()
venues["postal code"] += " "+ venues["country"]
#view values to confirm appropriate data before continuing build
venues


,vendor name,price,region,country,postal code,sector
2,Hart & Main,10656.0,Los Angeles,US,91321 US,Banquet Halls
3,Quiet Cannon,6985.0,Los Angeles,US,90640 US,Banquet Halls
4,Pomona Valley Mining Company,7401.0,Los Angeles,US,91768 US,Banquet Halls
5,Vertigo Event Venue,30000.0,Los Angeles,US,91202 US,Banquet Halls
6,Maravilla Gardens,16446.0,Santa Barbara,US,93012 US,Barn & Farm Weddings
...,...,...,...,...,...,...
178,The Kellogg House,9235.0,Los Angeles,US,91768 US,Historic Weddings
179,The Ebell of Los Angeles,29907.0,Los Angeles,US,90005 US,Historic Weddings
180,The Little White Church,3072.0,Los Angeles,US,91107 US,Church & Temple Weddings
181,Upland Events and Banquet Center,6338.0,Inland Empire,US,91786 US,Banquet Halls


In [36]:
#geocode zip codes for mapping

#initialize Nominatim geocoder instance
geolocator = Nominatim(user_agent = "wedding_venue")
#apply rate limiter per Nominatim usage rules
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#geocode locations and align with new column in dataframe
venues['location'] = venues['postal code'].apply(geocode)

In [37]:
#return latitude and longitude attributes from location object to standalone columns.
#not expected to repeat these functions, so, leveraged anonymous functions for the operation.
venues['latitude'] = venues['location'].apply(lambda instance: instance.latitude if instance else None)
venues['longitude'] = venues['location'].apply(lambda instance: instance.longitude if instance else None)

In [38]:
#remove location column from dataframe
venues=venues.drop('location', axis=1)

In [41]:
#initialize app using Dash framework
app = Dash(__name__)

'''
Build out html layout using a Div for the title, dropdown option to select sectors for presentation, table to show details,
a row in bootstrap to contain the histogram graph and scatter map plot of locations in LA
'''
app.layout = html.Div([
    html.Div(children='Wedding Venue Price Dashboard'),
    dcc.Dropdown(id='controls', options=[{'label': i, 'value': i} for i in venues['sector'].unique()]),
    dash_table.DataTable(id='table',page_size=6),
    dbc.Row([
        dbc.Col(dcc.Graph(figure={}, id='graph'), md=6),
        dbc.Col(dcc.Graph(figure={}, id='map'), md=6)
    ])
])

#build callback features to control input values via dropdown, and output to the table, graph, and map
@app.callback(
    [Output(component_id='graph', component_property='figure'),
    Output(component_id='table', component_property='data'),
    Output(component_id='map', component_property='figure')],
    Input(component_id='controls', component_property='value')
)

#run function upon callback to update the table, graph, and map 
def update_hist_and_table(sector):
    hist = px.histogram(venues[venues['sector'] == sector], x='price', labels={'price': 'Venue Price'})
    table = venues[venues['sector'] == sector].to_dict('records')
    fig = px.scatter_mapbox(venues[venues['sector']==sector], lat='latitude', lon='longitude', hover_name='vendor name', 
                            hover_data = ['price'])
    fig.update_layout(mapbox_style="stamen-toner")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_traces(marker=dict(size=15))
    return hist, table, fig

#run application
if __name__ == '__main__':
    app.run_server(debug=True)
